### 2nd -Order Language Model

p(w(t)|w(t-1),w(t-2))

Train a model , 
Generate new Phrases

## Data  
- Collection of Robert Frost Poems  
- Text is just a bunch of poems concatenated  

In [3]:
import numpy as np 
import string 

Initially we need to find following  
 - Initial Distribution of first word 
 - Second word distribution(won't have 2 previous words) 
 - End of line distribution(w(t-2),w(t-1) $\rightarrow$ END)

In [4]:
initial = {} #pdf for start of the phrase
second_word = {} #pdf for second_word of the phrase
transitions = {}  # Dictionary for second order transitions 

'''Removes punctuation from a dict''' 
translator = str.maketrans({key: None for key in string.punctuation})
def remove_punctuation(s):
    return s.translate(translator)  

def add2dict(d,k,v):
    if k not in d: d[k]=[]
    d[k].append(v)


for line in open("./data/second_order_markov/robert_frost.txt"):
    '''Tokenizing each sentence and removing punctuation'''
    tokens =remove_punctuation(line.rstrip().lower()).split() 
    T = len(tokens)
    for i in range(T):
        t=tokens[i]
        if i==0 : initial[t] = initial.get(t,0.)+1  #First Word
        else: 
            t_1= tokens[i-1]
            if i==T-1:
                add2dict(transitions,(t_1,t),'END') #Need to add for tranisition 
            if i==1:
                add2dict(second_word,t_1,t)  #Now add to second_word
            else:
                t_2 = tokens[i-2]
                add2dict(transitions,(t_2,t_1),t)  #Again transitiom

In [5]:
second_word

{'2o3': ['let'],
 'a': ['tree',
  'voice',
  'swamp',
  'tree',
  'barkless',
  'hole',
  'leak',
  'cliff',
  'likeness',
  'whole',
  'small',
  'moment',
  'bird',
  'single',
  'brush',
  'piercing',
  'governor',
  'rockstrewn',
  'bead',
  'little',
  'shelfs',
  'chimney',
  'few',
  'brook',
  'broken',
  'note',
  'friend',
  'bill',
  'winter',
  'featherhammer'],
 'about': ['where'],
 'according': ['as'],
 'across': ['each'],
 'after': ['the'],
 'against': ['the'],
 'alias': ['john'],
 'all': ['four', 'is', 'the', 'claiming', 'fresh', 'that', 'is', 'we'],
 'almost': ['the'],
 'always': ['the'],
 'among': ['the', 'the', 'unearthed'],
 'an': ['hour'],
 'ancestral': ['memories'],
 'and': ['sorry',
  'be',
  'looked',
  'having',
  'both',
  'that',
  'miles',
  'miles',
  'would',
  'dropped',
  'further',
  'when',
  'tell',
  'the',
  'caught',
  'put',
  'threw',
  'birds',
  'suddenly',
  'scurf',
  'sorry',
  'since',
  'whats',
  'tell',
  'many',
  'blew',
  'stamped',
 

In [6]:
#normalize the initial distributions 
initial_total = sum(initial.values()) 
for t,c in initial.items():
    initial[t] = c/initial_total  
    

In [7]:
#Normalising Transition and second dictionary 
def list2pdict(ts):
    d= {} 
    n = len(ts) 
    for t in ts:
        d[t] = d.get(t,0.) +1 
    for t,c in d.items():
        d[t] = c/n 
    return d 
    
for t_1,ts in second_word.items():
    second_word[t_1] = list2pdict(ts) 

for k,ts in transitions.items():
    transitions[k] = list2pdict(ts)

In [8]:
transitions

{('the', 'desert'): {'END': 1.0},
 ('each', 'other'): {'END': 0.4, 'and': 0.2, 'both': 0.2, 'i': 0.2},
 ('still', 'you'): {'mustnt': 1.0},
 ('mormon', 'settlement'): {'END': 1.0},
 ('own', 'farm'): {'hes': 1.0},
 ('we', 'followed'): {'this': 1.0},
 ('them', 'out'): {'END': 1.0},
 ('slumbering', 'village'): {'street': 1.0},
 ('no', 'stay'): {'unless': 1.0},
 ('run', 'down'): {'this': 1.0},
 ('or', 'slower'): {'as': 1.0},
 ('off', 'this'): {'mountain': 1.0},
 ('take', 'the'): {'polish': 0.5, 'timber': 0.5},
 ('strains', 'been'): {'too': 1.0},
 ('the', 'help'): {'END': 1.0},
 ('hue', 'to'): {'hold': 1.0},
 ('youd', 'best'): {'tempt': 1.0},
 ('shes', 'bad'): {'thats': 1.0},
 ('at', 'the'): {'door': 0.14285714285714285,
  'end': 0.14285714285714285,
  'kitchen': 0.14285714285714285,
  'level': 0.14285714285714285,
  'stake': 0.14285714285714285,
  'time': 0.14285714285714285,
  'waterside': 0.14285714285714285},
 ('less', 'traveled'): {'by': 1.0},
 ('situation', 'was'): {'like': 1.0},
 ('it

In [9]:
second_word

{'2o3': {'let': 1.0},
 'a': {'barkless': 0.03333333333333333,
  'bead': 0.03333333333333333,
  'bill': 0.03333333333333333,
  'bird': 0.03333333333333333,
  'broken': 0.03333333333333333,
  'brook': 0.03333333333333333,
  'brush': 0.03333333333333333,
  'chimney': 0.03333333333333333,
  'cliff': 0.03333333333333333,
  'featherhammer': 0.03333333333333333,
  'few': 0.03333333333333333,
  'friend': 0.03333333333333333,
  'governor': 0.03333333333333333,
  'hole': 0.03333333333333333,
  'leak': 0.03333333333333333,
  'likeness': 0.03333333333333333,
  'little': 0.03333333333333333,
  'moment': 0.03333333333333333,
  'note': 0.03333333333333333,
  'piercing': 0.03333333333333333,
  'rockstrewn': 0.03333333333333333,
  'shelfs': 0.03333333333333333,
  'single': 0.03333333333333333,
  'small': 0.03333333333333333,
  'swamp': 0.03333333333333333,
  'tree': 0.06666666666666667,
  'voice': 0.03333333333333333,
  'whole': 0.03333333333333333,
  'winter': 0.03333333333333333},
 'about': {'where':

In [55]:
"Function to return a sample word from dictionary"
def sample_word(d):
    p0 = np.random.random()
    cumulative =0 
    for t,p in d.items():
        cumulative +=p 
        if p0 < cumulative: 
            return t 
    assert(False) 

In [60]:
"Function to generate Random Sentences"
def generate():
    for i in range(4): 
        sentence=[] 
        w0 = sample_word(initial)
        sentence.append(w0)  

        w1 = sample_word(second_word[w0])
        sentence.append(w1) 

        while True: 
            w2 = sample_word(transitions[(w0,w1)])
            if w2 =='END':
                break
            sentence.append(w2)
            w0=w1
            w1=w2 
        print(' '.join(sentence)) 

generate()
        

last night was one of them put together
till well toward noon when the heat
except it seemed the poetesss life
to shut you up ill tell you what you show me you remember


In [19]:
line = "Two roads diverged ! in a yellow wood ,"
print(line.rstrip().lower()) 
print(remove_punctuation(line.rstrip().lower()))
print(remove_punctuation(line.rstrip().lower()).split()) 

two roads diverged ! in a yellow wood ,
two roads diverged  in a yellow wood 
['two', 'roads', 'diverged', 'in', 'a', 'yellow', 'wood']
